In [ ]:
"""
Purpose: To generate those same dataframes
that forrest did to provide us with some validation numbers

"""

In [3]:
from os import sys
sys.path.append("/meshAfterParty/meshAfterParty/")
import datajoint as dj
import datajoint_utils as du

minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-28 18:02:16,517 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-28 18:02:16,518 - settings - Setting database.user to celiib
INFO - 2021-04-28 18:02:16,519 - settings - Setting database.password to newceliipass
INFO - 2021-04-28 18:02:16,537 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-28 18:02:16,538 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-28 18:02:16,547 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-28 18:02:16,863 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-28 18:02:16,886 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-28 18:02:16,888 - settings - Setting database.user to celiib
INFO - 2021-04-28 18:02:16,889 - settings - Setting database.password to newceliipass
INFO - 2021-04-28 18:02:16,889 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-28 18:02:16,890 - settings - Setting database.user to celiib
INFO - 2021-04-28 18:02:16,891 - settings - Setting database.password to newceliipass
INFO - 2021-04-28 18:02:16,891 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-28 18:02:17,255 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-28 18:02:17,294 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-28 18:02:17,295 - settings - Setting database.user to celiib
INFO - 2021-04-28 18:02:17,296 - settings - Setting database.pa

In [5]:
m65 = minnie

In [6]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [ ]:
aug_1_seg_ids,aug_1_nucl_ids = final_nucs.fetch("segment_id","nucleus_id")

# His Old Code

In [ ]:
ds=[]
for root_id, gdf in tqdm.tqdm(prf_outputs.groupby('pt_root_id')):
    syn_svid=gdf.pre_pt_supervoxel_id.values
    for time in times:
        d={}
        rootIds=client.chunkedgraph.get_roots(syn_svid, timestamp=time)
        roots, counts = np.unique(rootIds, return_counts=True)
        p = counts/np.sum(counts)
        d['pmax'] = counts[np.argmax(counts)]
        d['nfrag'] = len(roots)
        d['entropy'] = stats.entropy(p)
        d['time']=time
        d['root_id']=root_id
        ds.append(d)

In [ ]:
def get_input_output_df(aug_seg_id,  n_chunks=50, timestamp=datetime.datetime(2020, 8, 1, 1, 0, 0)):
    svids=client.chunkedgraph.get_leaves(aug_seg_id)
    chunks = np.array_split(svids,n_chunks)
    #need to break the supervoxels up into chunks to query
    out_syn_dfs=[]
    for chunk in tqdm.tqdm(chunks):
        dft = client.materialize.query_table('synapses_pni_2',
                                           filter_in_dict={'pre_pt_supervoxel_id':chunk})
        out_syn_dfs.append(dft)
    out_syn_df = pd.concat(out_syn_dfs)
    out_syn_df['pre_pt_root_id_Aug']=aug_seg_id
    post_sv_ids=out_syn_df.post_pt_supervoxel_id.values
    post_root_ids_aug = client.chunkedgraph.get_roots(post_sv_ids, timestamp=august_timestamp)
    out_syn_df['post_pt_root_id_Aug']=post_root_ids_aug
    #  now the same for inputs
    in_syn_dfs=[]
    for chunk in tqdm.tqdm(chunks):
        dft = client.materialize.query_table('synapses_pni_2',
                                           filter_in_dict={'post_pt_supervoxel_id':chunk})
        in_syn_dfs.append(dft)
    in_syn_df = pd.concat(in_syn_dfs)
    in_syn_df['post_pt_root_id_Aug']=aug_seg_id
    pre_sv_ids=in_syn_df.pre_pt_supervoxel_id.values
    pre_root_ids_aug = client.chunkedgraph.get_roots(pre_sv_ids, timestamp=timestamp)
    in_syn_df['pre_pt_root_id_Aug']=pre_root_ids_aug
    return pd.concat([out_syn_df, in_syn_df])

In [ ]:
overlap = pd.merge(nuc_df, aug_nuc_df, left_on='id', right_on='flat_nuc_id', how='inner')
overlap.query('pt_root_id==root_id')

In [ ]:
nuc_df = client.materialize.query_table('nucleus_neuron_svm',
                                       filter_equal_dict={'cell_type':'neuron'},
                                       filter_out_dict={'pt_root_id':[0]})